# SQL Server

In [ ]:
# Import the required library
import pyodbc
import pandas as pd

# Database connection details for SQL Server
sql_server_connection_string = (
    'DRIVER={ODBC Driver 18 for SQL Server};' # Change the driver version if needed
    'SERVER=xxxxxxxx;'  # Change to your server
    'DATABASE=XXX;'          # Change to your database
    'UID=XXXXX;'                 # Change to your username
    'PWD=XXXXXXX;'        # Change to your password
    'TrustServerCertificate=yes;'
)

# Step 2: Get the table description using the DESCRIBE command
# Function to retrieve column names and data types from the MSSQL table
def get_table_schema(cursor, table_name):
    query = f"""
    SELECT COLUMN_NAME, DATA_TYPE, CHARACTER_MAXIMUM_LENGTH 
    FROM INFORMATION_SCHEMA.COLUMNS
    WHERE TABLE_NAME = '{table_name}'
    """
    cursor.execute(query)
    return cursor.fetchall()


# Table Name
table_name = 'XXXXXXXXXX'  # Replace with your table name

# Get the table schema (column names and data types)
with pyodbc.connect(sql_server_connection_string) as sql_conn:
    cursor = sql_conn.cursor()
    table_schema = get_table_schema(cursor, table_name)
    print("-------------------------------- Table Schema --------------------------------")
    print("Table Schema (Column Names and Data Types):")
    for column in table_schema:
        print(f"{column[0]}: {column[1]}({column[2]})")  # Column name, data type, and max

# Create automatic function to create SQL query to create table
def create_table_query(table_name, table_schema):
    query = f"CREATE TABLE {table_name} (\n"
    for column in table_schema:
        column_name = column[0]
        data_type = column[1]
        max_length = column[2]
        query += f"    {column_name} {data_type}"
        if max_length is not None:
            query += f"({max_length})"
        query += ",\n"
    query = query.rstrip(",\n")  # Remove the last comma and newline
    query += "\n);"
    return query

# Generate the CREATE TABLE query
create_table_query_str = create_table_query(table_name, table_schema)
print("\n------------------------- SQL Query to Create Table ---------------------------")
print("CREATE TABLE Query:")
print(create_table_query_str)

-------------------------------- Table Schema --------------------------------
Table Schema (Column Names and Data Types):
Check1: nvarchar(50)
Account: nvarchar(50)
Customer_Code: nvarchar(50)
Type: nvarchar(50)
Nama: nvarchar(250)
Search_name: nvarchar(250)
Customer_group: nvarchar(50)
Language: nvarchar(50)
Street: nvarchar(250)

------------------------- SQL Query to Create Table ---------------------------
CREATE TABLE Query:
CREATE TABLE mapping_customer (
    Check1 nvarchar(50),
    Account nvarchar(50),
    Customer_Code nvarchar(50),
    Type nvarchar(50),
    Nama nvarchar(250),
    Search_name nvarchar(250),
    Customer_group nvarchar(50),
    Language nvarchar(50),
    Street nvarchar(250)
);


# MySQL

In [ ]:
import mysql.connector
import re

# Function to check for non-ASCII characters or symbols in a string
def contains_non_ascii_or_symbols(data):
    # Check for non-ASCII characters or any non-alphanumeric character (symbol)
    return bool(re.search(r'[^a-zA-Z0-9]', data))

# Step 1: Connect to the MySQL database
connection = mysql.connector.connect(
    host="xxxxxxxx",  # Your MySQL server host
    user="xxxxxxxx",        # Your MySQL username
    password="xxxxxx", # Your MySQL password
    database="xxxxxxx"  # Your MySQL database name
)

# Step 2: Get the table description using the DESCRIBE command
table_name = "XXXXXXX"  # Name of the new table
cursor = connection.cursor()
cursor.execute(f"DESCRIBE {table_name}")
table_description = cursor.fetchall()

# Step 3: Construct the CREATE TABLE query based on the table description
create_table_query = f"CREATE TABLE {table_name} (\n"

for column in table_description:
    column_name = column[0]  # Column name
    column_type = column[1]  # Column type
    
    # If the column is of type VARCHAR, check if it contains non-ASCII characters or symbols
    if "varchar" in column_type.lower():
        # Check the actual data for non-ASCII characters or symbols
        cursor.execute(f"SELECT `{column_name}` FROM {table_name} LIMIT 1000")  # Limit to 1000 rows for efficiency
        rows = cursor.fetchall()

        needs_nvarchar = False
        for row in rows:
            if row[0] and contains_non_ascii_or_symbols(str(row[0])):  # Convert the data to string and check
                needs_nvarchar = True
                break
        
        # If non-ASCII or symbols are found, switch to NVARCHAR
        if needs_nvarchar:
            column_type = column_type.replace("varchar", "NVARCHAR")

    create_table_query += f"  {column_name} {column_type},\n"

# Remove the last comma and add closing bracket
create_table_query = create_table_query.rstrip(",\n") + "\n);"

# Step 4: Close the cursor and connection
cursor.close()
connection.close()

# Step 5: Print the CREATE TABLE query
print(create_table_query)

CREATE TABLE Master_Kode_masuk (
  kode_masuk varchar(145),
  kode_mitra NVARCHAR(45),
  Status_aktif varchar(1),
  Tgl_awal date,
  tgl_akhir date,
  Keterangan varchar(145),
  kode_coop varchar(45),
  Lantai NVARCHAR(45),
  Nama_farm NVARCHAR(45),
  item_pre_starter NVARCHAR(45),
  item_starter NVARCHAR(45),
  item_finisher NVARCHAR(45),
  Aktif_BE_simulated varchar(45),
  chickin_be_simulated int(11),
  Harga_DOC decimal(18,2),
  Total_Value_Panen decimal(18,2),
  Biaya_Electric decimal(18,2),
  Biaya_water decimal(18,2),
  Biaya_Labour decimal(18,2),
  Biaya_other decimal(18,2)
);


# BigQuery

In [3]:
import pandas as pd
from google.cloud import bigquery
import os

# Mapping BigQuery data types to SQL Server data types
data_type_mapping = {
    "STRING": "VARCHAR(255)",
    "INT64": "BIGINT",
    "FLOAT64": "FLOAT",
    "BOOL": "BIT",
    "NUMERIC": "DECIMAL(38, 9)",
    "BIGNUMERIC": "DECIMAL(38, 9)",
    "TIMESTAMP": "DATETIME",
    "DATE": "DATE",
    "TIME": "TIME"
}

# Function to retrieve column names and data types from BigQuery
def get_bq_table_schema(client, dataset_id, table_name):
    query = f"""
    SELECT column_name, data_type
    FROM `{dataset_id}.INFORMATION_SCHEMA.COLUMNS`
    WHERE table_name = '{table_name}'
    """
    query_job = client.query(query)
    return query_job.result()

# Function to generate CREATE TABLE query for SQL Server
def create_sql_server_table_query(table_name, table_schema):
    query = f"CREATE TABLE {table_name} (\n"
    for column in table_schema:
        column_name = column["column_name"]
        bq_type = column["data_type"].upper()
        sql_server_type = data_type_mapping.get(bq_type, "VARCHAR(255)")  # Default to VARCHAR(255)
        query += f"    {column_name} {sql_server_type},\n"
    query = query.rstrip(",\n")  # Remove the last comma and newline
    query += "\n);"
    return query

# Set Google Cloud credentials
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "service-account.json"

# Initialize BigQuery client
client = bigquery.Client()

dataset_id = "SICE"  # Replace with your dataset
bq_table_name = "FactTarget"  # Replace with your table name

# Get the table schema from BigQuery
table_schema = get_bq_table_schema(client, dataset_id, bq_table_name)

# Generate the CREATE TABLE query for SQL Server
create_sql_server_query_str = create_sql_server_table_query(bq_table_name, table_schema)
print("\n------------------------- SQL Server CREATE TABLE Query ---------------------------")
print(create_sql_server_query_str)


------------------------- SQL Server CREATE TABLE Query ---------------------------
CREATE TABLE FactTarget (
    DateKey VARCHAR(255),
    ScenarioId VARCHAR(255),
    BuidId VARCHAR(255),
    CustomerId VARCHAR(255),
    ItemId VARCHAR(255),
    AreaId VARCHAR(255),
    PersonelId VARCHAR(255),
    Weight FLOAT,
    PriceUnit FLOAT,
    Amount FLOAT,
    Tanggal_key DATE,
    CustomerId_key VARCHAR(255),
    ItemId_key VARCHAR(255),
    Category_Product VARCHAR(255),
    Product_Brand VARCHAR(255),
    Product_Focus VARCHAR(255)
);
